<font color='red'>注：此处是文档第218页</font>

# 高级：制定动态决策和BI-LSTM CRF

## 1.动态与静态深度学习工具包
Pytorch是一种**动态**神经网络套件。另一个动态套件的例子是[Dynet](https://github.com/clab/dynet)（我之所以提到这一点，因为与Pytorch和Dynet一起使用是相似的。如果你在Dynet中看到一个例子，它可能会帮助你在Pytorch中实现它）。相反的是**静态**工具包，其中包括Theano，Keras，TensorFlow等。核心区别如下： 
* 在静态工具包中，您可以定义一次计算图，对其进行编译，然后将实例流式传输给它。 
* 在动态工具包中，为每个实例定义计算图。它永远不会被编译并且是即时执行的。

在没有很多经验的情况下，很难理解其中的差异。一个例子是假设我们想要构建一个深层组成解析器。假设我们的模型大致涉及以下步骤： 
* 我们自底向上地建造树 
* 标记根节点（句子的单词）
* 从那里，使用神经网络和单词的嵌入来找到形成组成部分的组合。

每当你形成一个新的成分时，使用某种技术来嵌入成分。在这种情况下，我们的网络架构将完全取决于输入句子。在“The green cat scratched the wall”一句中，在模型中的某个点上，我们想要结合跨度$(i,j,r)=(1.3,NP)$（即，NP 组成部分跨越单词1到单词3，在这种情况下是“The green cat”）。

然而，另一句话可能是“Somewhere, the big fat cat scratched the wall”。在这句话中，我们希望在某个时刻形成组成$(2,4,NP)$。我们想要形成的成分将取决于实例。如果我们只编译计算图一次，就像在静态工具包中那样，但编写这个逻辑将非常困难或者说是不可能的。但是，在动态工具包中，不仅有1个预定义的计算图。每个实例都可以有一个新的计算图，所以这个问题就消失了。

动态工具包还具有易于调试和代码更接近宿主语言的优点（我的意思是Pytorch和Dynet看起来更像是比Keras或Theano更实际的Python代码）。

## 2.Bi-LSTM条件随机场讨论
对于本节，我们将看到用于命名实体识别的Bi-LSTM条件随机场的完整复杂示例。虽然上面的LSTM标记符通常足以用于词性标注，但是像CRF这样的序列模型对于NER上的强大性能非常重要。CRF，虽然这个名字听起来很可怕，但所有模型都是CRF，在LSTM中提供了这些功能。CRF是一个高级模型，比本教程中的任何早期模型复杂得多。如果你想跳过它，也可以。要查看您是否准备好，请查看是否可以：
+ 在步骤i中为标记k写出维特比变量的递归。
+ 修改上述重复以计算转发变量。
+ 再次修改上面的重复计算以计算日志空间中的转发变量（提示：log-sum-exp）

如果你可以做这三件事，你应该能够理解下面的代码。回想一下，CRF计算条件概率。设$y$为标签序列，$x$为字的输入序列。然后我们计算
$$p(y|x)=\frac{exp(Score(x,y))}{\sum_{y^\prime}exp(Score(x,y))}$$
通过定义一些对数电位$log\psi_i(x,y)$来确定得分:
$$Score(x,y)=\sum_{i}log\psi_i(x,y)$$
为了使分区功能易于处理，电位必须仅查看局部特征。

在Bi-LSTM CRF中，我们定义了两种潜力：发射和过渡。索引$i$处的单词的发射电位来自时间步长$i$处的Bi-LSTM的隐藏状态。转换分数存储在$T$矩阵$P$中，其中$T$是标记集。在我们的实现中，$P_{j_{y}k}$是从标签$k$转换到标签$j$的分数。所以：
$$Score(x,y)=\sum_{i}log\psi_{EMIT}(y_i\rightarrow x_i)+log\psi_{TRANS}(y_{i-1}\rightarrow y_i)=\sum_{i}h_i[y_i]+P_{y_i},y_i-1$$
在第二个表达式中，我们将标记视为分配了唯一的非负索引。

如果上面的讨论过于简短，你可以查看[这个](http://www.cs.columbia.edu/~mcollins/crf.pdf)，是迈克尔柯林斯写的关于CRF的文章。

## 3.实现说明
下面的示例实现了日志空间中的前向算法来计算分区函数，以及用于解码的维特比算法。反向传播将自动为我们计算梯度。我们不需要手工做任何事情。

这个实现并未优化。如果您了解发生了什么，您可能会很快发现在前向算法中迭代下一个标记可能是在一个大的操作中完成的。我想编码更具可读性。 如果您想进行相关更改，可以将此标记器用于实际任务。